## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [11]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [12]:
with open("data/wordsim_similarity_goldstandard.txt", encoding="utf-8") as rf:
  triples = [line.strip().split("\t") for line in rf.readlines()]
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

In [13]:
import csv

score_map = {}
with open('data/Task_4_sample_13.csv', 'r', newline='') as csvfile:
  spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
  first = True
  for w1, w2, score in spamreader:
      if first:
          first = False
          continue
      score_map[(w1, w2)] = float(score)

In [14]:
score_map

{('bird', 'cock'): 7.1,
 ('start', 'year'): 4.06,
 ('vodka', 'brandy'): 8.13,
 ('bread', 'butter'): 6.19,
 ('cup', 'article'): 2.4,
 ('street', 'place'): 6.44,
 ('stock', 'phone'): 1.62,
 ('street', 'children'): 4.94,
 ('direction', 'combination'): 2.25,
 ('cemetery', 'woodland'): 2.08,
 ('dollar', 'yen'): 7.78,
 ('Harvard', 'Yale'): 8.13,
 ('month', 'hotel'): 1.81,
 ('delay', 'news'): 3.31,
 ('announcement', 'production'): 3.38,
 ('plane', 'car'): 5.77,
 ('mile', 'kilometer'): 8.66,
 ('championship', 'tournament'): 8.36,
 ('space', 'chemistry'): 4.88,
 ('minority', 'peace'): 3.69,
 ('food', 'fruit'): 7.52,
 ('coast', 'shore'): 9.1,
 ('precedent', 'collection'): 2.5,
 ('rooster', 'voyage'): 0.62,
 ('monk', 'oracle'): 5.0,
 ('cup', 'entity'): 2.15,
 ('midday', 'noon'): 9.29,
 ('school', 'center'): 3.44,
 ('street', 'block'): 6.88,
 ('asylum', 'madhouse'): 8.87,
 ('opera', 'performance'): 6.88,
 ('seafood', 'lobster'): 8.7,
 ('board', 'recommendation'): 4.47,
 ('precedent', 'example'): 5

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [15]:
for w1, w2 in list(score_map)[:2]:
  
  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: bird-cock
Ground truth score: 7.10

Path: 0.062 
wup: 0.211 
shortest_path: 15.000

Words: start-year
Ground truth score: 4.06

Path: 0.100 
wup: 0.308 
shortest_path: 9.000


Вычисляем для всех пар несколько оценок

In [16]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list, lch_list = [], [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)
    
    lch = max([item1.lch_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    lch_list.append(path)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

## Вычисляем ранговую корреляцию Спирмена

In [17]:
from scipy.stats import spearmanr

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)

coef, p = spearmanr(lch_list, true_list)
print("lch Spearman R: %.4f" % coef)

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)


path Spearman R: 0.6254
lch Spearman R: 0.6254
wup  Spearman R: 0.6343


При помощи метода hyponyms() найдите количество гипонимов для синсета dollar.n.01, а также при помощи метода name() найдите значение первого в гипонима из списка.

In [18]:
synset = "dollar.n.01"
hyponyms = wn.synset(synset).hyponyms()
len(hyponyms)

23

In [19]:
sorted(hyponyms)[0].name()

'australian_dollar.n.01'